<a href="https://colab.research.google.com/github/Accenture-1a-BTT/accenture-1a-team-project/blob/test-Siying/FakeNews_Accenture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Exploratory Data Analysis Data Cleaning

## Single Dataset Inspection and Cleaning

Load Data Sets and Save it as a Pandas DataFrame

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
folder_path = '/content/drive/My Drive/Break Through Tech/AI Studio/Google Colab/Data Set/'
true_file = os.path.join(folder_path, 'True.csv')
fake_file = os.path.join(folder_path, 'Fake.csv')
true_df = pd.read_csv(true_file,header=0)
fake_df = pd.read_csv(fake_file,header=0)

Basic Inspection

In [5]:
# Check the data set size
print("True news data set size: ", true_df.size)
print("Fake news data set size: ", fake_df.size)

True news data set size:  85668
Fake news data set size:  93924


In [6]:
# Check Columns of Data Set
print("Columns of true news data set: ", list(true_df.columns))
print("Columns of fake news data set: ", list(fake_df.columns))

Columns of true news data set:  ['title', 'text', 'subject', 'date']
Columns of fake news data set:  ['title', 'text', 'subject', 'date']


In [7]:
# Check Data Types
print("Data types of true news data set: ")
print(true_df.dtypes)
print("Data types of fake news data set: ")
print(fake_df.dtypes)

Data types of true news data set: 
title      object
text       object
subject    object
date       object
dtype: object
Data types of fake news data set: 
title      object
text       object
subject    object
date       object
dtype: object


In [8]:
print("First 10 rows of True News DataFrame: ")
true_df.head(10)

First 10 rows of True News DataFrame: 


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017"
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017"
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017"


In [9]:
print("First 10 rows of Fake News DataFrame: ")
fake_df.head(10)

First 10 rows of Fake News DataFrame: 


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017"
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017"
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017"
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017"
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017"


Check missing values in each dataset

In [14]:
print("Null value on each data set: ")
print("True news file:\n", true_df.isnull().sum())
print("Fake news file:\n", fake_df.isnull().sum())
# Siying Comment:
# We know from the inspection that there is no null value on the data set so we don't need to worry about the null value.

print("\nEmpty strings check:")
print("True news empty text:", (true_df['text'].str.strip() == '').sum())
print("Fake news empty text:", (fake_df['text'].str.strip() == '').sum())
# Siying Comment:
# However, there is no null value in the dataset does not means there is no empty string on the data set.
# And here we did find some rows with empty string in the 'text' feature.
# My suggestion is we can simply delete the rows with empty string since it will not provide any useful information for the model.

print("\nVery short text:")
print("True news < 10 chars:", (true_df['text'].str.len() < 10).sum())
print("Fake news < 10 chars:", (fake_df['text'].str.len() < 10).sum())
# Siying Comment:
# Also, I found some data set with very short string length, I'm quite worried about if it will provide us sufficient information to train the model.
# Since Chinyere will be focus on punctuation and sentence length, I think I will leave this question to her that how should we handle this, keep them or drop them.
# # For now, I will remove texts with less than 10 characters since they cannot provide meaningful features for us

# Siying Summary:
# Fake news dataset has 630 empty strings and 651 very short texts. These numbers are very close, which indicates that the most of short texts are actually empty strings.
# Also, notice the disparity in terms of empty strings and short texts:
# - True news: 1 empty string, 1 very short text
#             (can indicates that the very short text is the empty string, which means that true news does not has short text problems)
# - Fake news: 630 empty string, 651 very short text, which means 21 very short texts
# It indicates an important differences between real and fake news sources, which could be a distinguishing feature that we can pay attention to.

Null value on each data set: 
True news file:
 title      0
text       0
subject    0
date       0
dtype: int64
Fake news file:
 title      0
text       0
subject    0
date       0
dtype: int64

Empty strings check:
True news empty text: 1
Fake news empty text: 630

Very short text:
True news < 10 chars: 1
Fake news < 10 chars: 651


Remove empty string and very short text from each dataset

In [21]:
# Remove empty string and very short text from true news data set
true_nonempty_df = true_df[(true_df['text'].str.strip() != '') & (true_df['text'].str.len() >= 10)].copy()
# Remove empty string and very short text from fake news data set
fake_nonempty_df = fake_df[(fake_df['text'].str.strip() != '') & (fake_df['text'].str.len() >= 10)].copy()

print("After removing empty string and very short text")
print(f"Ture news size: {len(true_nonempty_df)}(removed {len(true_df) - len(true_nonempty_df)} rows)")
print(f"Fake news size: {len(fake_nonempty_df)}(removed {len(fake_df) - len(fake_nonempty_df)} rows)")

After removing empty string and very short text
Ture news size: 21416(removed 1 rows)
Fake news size: 22830(removed 651 rows)


Check for completely duplicate rows in each dataset

In [22]:
# Check for completely duplicate articles (which means includes title, text, subject, and date)
print("True Dataframe -- Number of duplicate rows:")
print(true_nonempty_df.duplicated().sum())
print("Fake Dataframe -- Number of duplicate rows:")
print(fake_nonempty_df.duplicated().sum())

# Siying Comment:
# Found 206 duplicate rows in real news and 3 in fake news datasets. These are completely identical articles (same title, text, subject, and date).
# Duplicate rows can lead to:
# 1. overfitting,
# 2. give undue weight to our model
# 3. bias on our feature analysis
# 4. more conputational cost
# Thus, I suggest that we should remove the duplicate rows to ensure our data is clean and unbiasd.

True Dataframe -- Number of duplicate rows:
206
Fake Dataframe -- Number of duplicate rows:
3


Remove completely duplicate rows from each dataset

In [25]:
# Remove completely duplicate articles
true_no_duplicates_df = true_nonempty_df.drop_duplicates().copy()
print(f"Remove {len(true_nonempty_df) - len(true_no_duplicates_df)} dulplicate rows")
fake_no_duplicates_df = fake_nonempty_df.drop_duplicates().copy()
print(f"Remove {len(fake_nonempty_df) - len(fake_no_duplicates_df)} dulplicate rows")

Remove 206 dulplicate rows
Remove 3 dulplicate rows


## Merge and cross-check

Add labels to each cleaned dataset

In [27]:
# Add labels before merging
true_no_duplicates_df['label'] = 'TRUE'
fake_no_duplicates_df['label'] = 'FAKE'

Merge datasets into combined dataframe

In [28]:
# Merge the cleaned data sets
merged_df = pd.concat([true_no_duplicates_df, fake_no_duplicates_df], ignore_index=True)

Merged Data Set Inspection

In [29]:
# Check Merged Data Sets
print("Combined dataset shape:", merged_df.shape)
print("Label counts:")
print(merged_df['label'].value_counts())

print("\nMerge Data Set Preview: ")
merged_df.head(10)

Combined dataset shape: (44037, 5)
Label counts:
label
FAKE    22827
TRUE    21210
Name: count, dtype: int64

Merge Data Set Preview: 


,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",TRUE
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",TRUE
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",TRUE
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",TRUE
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",TRUE
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017",TRUE
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017",TRUE
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017",TRUE
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017",TRUE
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017",TRUE


Check for conflicting labels (same text, different labels)

In [32]:
# Check if same text has conflict labels
# Groups all rows that have identical text content together, for each group, look at the 'label' column, count how many unique labels exist in each group
text_label_counts = merged_df.groupby('text')['label'].nunique()
# Only look at the groups that has more than 1 label
conflicting_texts = text_label_counts[text_label_counts > 1]

# If any conflict label found:
if len(conflicting_texts) > 0:
  for i, text in enumerate(conflicting_texts.index):
    print(f"\n--- CONFLICT {i+1} ---")
    conflicting_rows = merged_df[merged_df['text'] == text]

    print("Label counts for this article: ")
    print(conflicting_rows['label'].value_counts())

    print(f"Text: {text}")
else:
  print("No conflict labels found!")

# Siying Comment:
# Found no conflicting labels, which means there are no articles with the same text but got different label. It's nice that we don't need to worry about it.

No conflict labels found!


Check for duplicate text content (same text)

In [37]:
print("Number of articles with duplicate text content in data frame:")
print(merged_df['text'].duplicated().sum())

# Siying Comment:
# Found 5402 duplicated text on the data frame.
# These are identical text but with different title, text, subject, and date. They might be the articles that reposted by other media.
# Since keep these duplicated text might also cause bias and overfitting, Thus, I suggest remove these rows with duplicated text as well.

Number of articles with duplicate text content in data frame:
5402


Remove duplicate text content

In [39]:
merged_no_duplicates_df = merged_df['text'].drop_duplicates().copy()
print(f"Remove {len(merged_df) - len(merged_no_duplicates_df)} dulplicate rows")

Remove 5402 dulplicate rows
